# How to index Phenopackets with LinkML-Store





## Use pystow to download phenopackets

We will download from the Monarch Initiative [phenopacket-store](https://github.com/monarch-initiative/phenopacket-store)

In [1]:
import pandas as pd
import pystow
import yaml

path = pystow.ensure_untar("tmp", "phenopackets", url=" https://github.com/monarch-initiative/phenopacket-store/releases/latest/download/all_phenopackets.tgz")

In [2]:
# iterate over all *.json files in the phenopackets directory and parse to an object
# we will recursively walk the path using os.walk ( we don't worry about loading yet)
import os
import json
objs = []
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(".json"):
            with open(os.path.join(root, file)) as stream:
                obj = json.load(stream)
                objs.append(obj)
len(objs)

4876

## Creating a client and attaching to a database

First we will create a client as normal:

In [3]:
from linkml_store import Client

client = Client()

Next we'll attach to a MongoDB instance. this assumes you have one running already.

We will make a database called "phenopackets" and recreate it if it already exists

(note for people running this notebook locally - if you happen to have a database with this name in your current mongo instance it will be deleted!)

In [4]:
db = client.attach_database("mongodb://localhost:27017", "phenopackets", recreate_if_exists=True)

## Creating a collection

We'll create a simple test collection. The concept of collection in linkml-store maps directly to mongodb collections

In [5]:
collection = db.create_collection("main", recreate_if_exists=True)

## Inserting objects into the store

We'll use the standard `insert` method to insert the phenopackets into the collection. At this stage there is no explicit schema.

In [6]:
collection.insert(objs)

## Check contents

We can check the number of rows in the collection, to ensure everything was inserted correctly:

In [7]:
collection.find({}, limit=1).num_rows

4876

In [8]:
assert collection.find({}, limit=1).num_rows == len(objs)

Let's check with pandas just to make sure it looks as expected; we'll query for a specific OMIM disease:

In [9]:
qr = collection.find({"diseases.term.id": "OMIM:618499"}, limit=3)
qr.rows_dataframe

,id,subject,phenotypicFeatures,interpretations,diseases,metaData
0,PMID_28289718_Higgins-Patient-1,"{'id': 'Higgins-Patient-1', 'timeAtLastEncount...","[{'type': {'id': 'HP:0001714', 'label': 'Ventr...","[{'id': 'Higgins-Patient-1', 'progressStatus':...","[{'term': {'id': 'OMIM:618499', 'label': 'Noon...","{'created': '2024-03-28T11:11:48.590163946Z', ..."
1,PMID_31173466_Suzuki-Patient-1,"{'id': 'Suzuki-Patient-1', 'timeAtLastEncounte...","[{'type': {'id': 'HP:0001714', 'label': 'Ventr...","[{'id': 'Suzuki-Patient-1', 'progressStatus': ...","[{'term': {'id': 'OMIM:618499', 'label': 'Noon...","{'created': '2024-03-28T11:11:48.594725131Z', ..."
2,PMID_28289718_Higgins-Patient-2,"{'id': 'Higgins-Patient-2', 'timeAtLastEncount...","[{'type': {'id': 'HP:0001714', 'label': 'Ventr...","[{'id': 'Higgins-Patient-2', 'progressStatus':...","[{'term': {'id': 'OMIM:618499', 'label': 'Noon...","{'created': '2024-03-28T11:11:48.592718124Z', ..."


As expected, there are three rows with the OMIM disease 618499.

## Query faceting

We will now demonstrate faceted queries, allowing us to count the number of instances of different categorical values or categorical value combinations.

First we'll facet on the subject sex. We can use path notation, e.g. `subject.sex` here:

In [10]:
collection.query_facets({}, facet_columns=["subject.sex"])

{'subject.sex': [('MALE', 1807), ('FEMALE', 1564)]}

We can also facet by the disease name/label. We'll restrict this to the top 20

In [11]:
collection.query_facets({}, facet_columns=["diseases.term.label"], facet_limit=20)


{'diseases.term.label': [('Developmental and epileptic encephalopathy 4', 463),
  ('Developmental and epileptic encephalopathy 11', 342),
  ('KBG syndrome', 337),
  ('Leber congenital amaurosis 6', 191),
  ('Glass syndrome', 158),
  ('Holt-Oram syndrome', 103),
  ('Mitochondrial DNA depletion syndrome 13 (encephalomyopathic type)', 95),
  ('Neurodevelopmental disorder with coarse facies and mild distal skeletal abnormalities',
   73),
  ('Jacobsen syndrome', 69),
  ('Coffin-Siris syndrome 8', 65),
  ('Kabuki Syndrome 1', 65),
  ('Houge-Janssen syndrome 2', 60),
  ('ZTTK SYNDROME', 52),
  ('Greig cephalopolysyndactyly syndrome', 51),
  ('Seizures, benign familial infantile, 3', 51),
  ('Mitochondrial DNA depletion syndrome 6 (hepatocerebral type)', 50),
  ('Marfan syndrome', 50),
  ('Developmental delay, dysmorphic facies, and brain anomalies', 49),
  ('Loeys-Dietz syndrome 3', 49),
  ('Intellectual developmental disorder, autosomal dominant 21', 46)]}

In [12]:
collection.query_facets({}, facet_columns=["subject.timeAtLastEncounter.age.iso8601duration"], facet_limit=10)


{'subject.timeAtLastEncounter.age.iso8601duration': [('P4Y', 131),
  ('P3Y', 114),
  ('P6Y', 100),
  ('P5Y', 97),
  ('P2Y', 95),
  ('P7Y', 85),
  ('P10Y', 82),
  ('P9Y', 77),
  ('P8Y', 71)]}

In [13]:
collection.query_facets({}, facet_columns=["interpretations.diagnosis.genomicInterpretations.variantInterpretation.variationDescriptor.geneContext.symbol"], facet_limit=10)


{'interpretations.diagnosis.genomicInterpretations.variantInterpretation.variationDescriptor.geneContext.symbol': [('STXBP1',
   463),
  ('SCN2A', 393),
  ('ANKRD11', 337),
  ('RPGRIP1', 273),
  ('SATB2', 158),
  ('FBN1', 151),
  ('LMNA', 127),
  ('FBXL4', 117),
  ('TBX5', 103),
  ('SPTAN1', 85)]}

We can also facet on combinations:

In [14]:
fqr = collection.query_facets({}, facet_columns=[("subject.sex", "diseases.term.label")], facet_limit=20)
fqr


{('subject.sex', 'diseases.term.label'): [(('MALE', 'KBG syndrome'), 175),
  (('FEMALE', 'KBG syndrome'), 143),
  (('MALE', 'Glass syndrome'), 90),
  (('FEMALE', 'Glass syndrome'), 62),
  (('MALE',
    'Mitochondrial DNA depletion syndrome 13 (encephalomyopathic type)'),
   58),
  (('MALE',
    'Neurodevelopmental disorder with coarse facies and mild distal skeletal abnormalities'),
   54),
  (('FEMALE', 'Jacobsen syndrome'), 49),
  (('MALE', 'Coffin-Siris syndrome 8'), 37),
  (('FEMALE',
    'Mitochondrial DNA depletion syndrome 13 (encephalomyopathic type)'),
   37),
  (('FEMALE', 'Kabuki Syndrome 1'), 35),
  (('MALE', 'Houge-Janssen syndrome 2'), 32),
  (('MALE', 'Kabuki Syndrome 1'), 30),
  (('FEMALE', 'Developmental delay, dysmorphic facies, and brain anomalies'),
   29),
  (('MALE', 'Cardiac, facial, and digital anomalies with developmental delay'),
   28),
  (('FEMALE', 'Holt-Oram syndrome'), 28),
  (('MALE', 'Intellectual developmental disorder, autosomal dominant 21'), 28),
  

In [15]:
import pandas as pd
def fqr_as_dfs(fqr: dict):
    dfs = []
    for k, vs in fqr.items():
        rows = []
        for obj, count in vs:
            row = {}
            for col, val in zip(k, obj.values()):
                row[col] = val[0] if isinstance(val, list) else val
            row["count"] = count
            rows.append(row)
        df = pd.DataFrame(columns=list(k) + ["count"], data=rows)
        dfs.append(df)
    return dfs

fqr_as_dfs(fqr)[0]

AttributeError: 'tuple' object has no attribute 'values'

## Semantic Search

We will index phenopackets using a template that extracts the subject, phenotypic features and diseases.

First we will create a textualization template for a phenopacket. We will keep it minimal for simplicity - this doesn't include treatments, families, etc.

In [ ]:
template = """
subject: {{subject}}
phenotypes: {% for p in phenotypicFeatures %}{{p.type.label}}{% endfor %}
diseases: {% for d in diseases %}{{d.term.label}}{% endfor %}
"""

Next we will create an indexer using the template. This will use the Jinja2 syntax for templating.
We will also cache LLM embedding queries, so if we want to incrementally add new phenopackets we can avoid re-running the LLM embeddings calls.

In [ ]:
from linkml_store.index.implementations.llm_indexer import LLMIndexer

index = LLMIndexer(
    name="ppkt", 
    cached_embeddings_database="tmp/llm_pheno_cache.db",
    text_template=template,
    text_template_syntax="jinja2",
)

We can test the template on the first row of the collection:

In [ ]:
print(index.object_to_text(qr.rows[0]))

That looks as expected. We can now attach the indexer to the collection and index the collection:

In [ ]:
collection.attach_indexer(index, auto_index=True)

## Semantic Search

Let's query based on text criteria:

In [ ]:
qr = collection.search("patients with liver diseases")
qr.rows_dataframe[0:5]

Let's check the first one

In [ ]:
qr.ranked_rows[0]

We can combine semantic search with queries:

In [ ]:
qr = collection.search("patients with liver diseases", where={"subject.sex": "MALE"})
qr.rows_dataframe[0:5]

## Validation

Next we will demonstrate validation over a whole collection.

Currently validating depends on a LinkML schema - we have previously copied this schema into the test folder.
We will load the schema into the database object:

In [ ]:
db.load_schema_view("../../tests/input/schemas/phenopackets_linkml/phenopackets.yaml")

Quick sanity check to ensure that worked:

In [ ]:
list(db.schema_view.all_classes())[0:10]

In [ ]:
collection.metadata.type = "Phenopacket"

In [ ]:
from linkml_runtime.dumpers import yaml_dumper
for r in db.iter_validate_database():
    # known issue - https://github.com/monarch-initiative/phenopacket-store/issues/97
    if "is not of type 'integer'" in r.message:
        continue
    print(r.message[0:100])
    print(r)
    raise ValueError("Unexpected validation error")

## Command Line Usage

We can also use the command line for all of the above operations.

For example, feceted queries:

In [ ]:
!linkml-store -d mongodb://localhost:27017 -c main fq -S subject.sex

In [ ]:
!linkml-store -d mongodb://localhost:27017 -c main fq -S phenotypicFeatures.type.label -O yaml


In [ ]:
!linkml-store -d mongodb://localhost:27017 -c main fq -S diseases.term.label+subject.sex -O yaml
